In [1]:
import pandas as pd
from prophet import Prophet
from data_extraction.dummy_data_extractor import extract_dummy_data

In [ ]:
def facebook_pred(df, length_of_missing_data, data_logging_interval, dqStart):
    

    # Keep only the first two columns
    df = df.iloc[:, :2]

    df.set_index(df.columns[0], inplace=True, drop=True)
    
    df = df[df.index < dqStart]

    df.reset_index(inplace=True)

    # Rename columns
    df.columns = ['ds','y']


    df['ds'] = df['ds'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Initialize Prophet model with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive',  # Adjust based on data exploration
                         interval_width=0.95,          # Adjust prediction interval if needed
                         changepoint_prior_scale=0.001) # Tune based on data patterns

    # Fit the model
    model_temp.fit(df)

    # Number of predictions
    samples = int(length_of_missing_data / data_logging_interval) + 1

    # Create future DataFrame
    future_temp = model_temp.make_future_dataframe(periods=samples, freq='5T')

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)

    # Convert dq_start to timezone-naive
    dq_start = dqStart.strftime('%Y-%m-%d %H:%M:%S')

    
    # Filter predictions to start from dq_start
    predictions = forecast_temp[forecast_temp['ds'] >= dq_start][['ds', 'yhat']]

    # Set 'ds' as the index
    predictions.set_index('ds', inplace=True)

    return predictions

In [7]:
# Example usage:
master_table = extract_dummy_data("dummy_data")
df = master_table.at[0, "his"].iloc[:, :2]

# Extract values from the second row of master_table
length_of_missing_data = master_table.at[0, "dqDuration"]#pd.Timedelta(master_table.at[1, "dqDuration"])
data_logging_interval = master_table.at[0, "pointInterval"]#pd.Timedelta(master_table.at[1, "pointInterval"])
dqStart = master_table.at[0, "dqStart"]

# Call the function
predictions = facebook_pred(df, length_of_missing_data, data_logging_interval, dqStart)

# Display the resulting dataframe
# print(predictions.head())
predictions

KeyError: 1

------------

In [9]:
master_table.at[0,"his"]

,Timestamp,val,discharge_Chilled_Water_Cool_Valve_Cmd_AHU_01_B1,discharge_Fan_Vfd_Speed_Sensor_AHU_01_B1
0,2023-04-09 13:20:00+04:00,17.938112,29.293743,51.536274
1,2023-04-09 13:25:00+04:00,17.938112,29.293743,51.532143
2,2023-04-09 13:30:00+04:00,17.938112,29.293743,51.365505
3,2023-04-09 13:35:00+04:00,17.938112,29.293743,50.943623
4,2023-04-09 13:40:00+04:00,17.938112,29.293743,50.646545
...,...,...,...,...
9338,2023-05-11 23:30:00+04:00,NaN,30.918070,47.397423
9339,2023-05-11 23:35:00+04:00,NaN,29.926458,47.354992
9340,2023-05-11 23:40:00+04:00,NaN,29.926458,47.344307
9341,2023-05-11 23:45:00+04:00,NaN,28.917969,47.172913
